# Canny Benchmark

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append("../")

if False:
    # Enable the CUDA simulator
    os.environ["NUMBA_OPT"] = "0"
    os.environ["NUMBA_ENABLE_CUDASIM"] = "1"
    os.environ["NUMBA_CUDA_DEBUGINFO"] = "1"

import logging
import time
import pathlib
from pathlib import Path
import importlib.util
from importlib import reload
import math
from io import BytesIO
import unittest.mock as mock
import gc

import matplotlib as mpl
import matplotlib.pyplot as plt

from icecream import ic

import numpy as np
import pandas as pd
import scipy as sp
import numba
from numba import cuda

import cv2

from IPython.display import clear_output, display, Image
import ipywidgets as widgets
from PIL import Image as PILImage

from utils.benchmarking import time_function, time_line
from utils.setup_notebook import init_notebook, source_code_path_is_from_notebook
from utils.setup_logging import setup_logging

init_notebook()
setup_logging("DEBUG")


In [2]:
# Load canny implementations
dir_canny_impls = "./canny_impls"
files_canny_impls = [f for f in os.listdir(dir_canny_impls) if str(f).endswith(".py")]

if "canny_impls" not in locals():
    canny_impls = []


def load_module(file_path: str, dyn_modules: list):
    module_name = Path(file_path).stem
    module_string = f"{module_name}"

    already_loaded = module_string in sys.modules
    if already_loaded:
        with time_line(f"Reloading {module_name}"):
            module = sys.modules[module_string]
            reload(module)
        return module

    with time_line(f"Loading {module_name}"):
        folder = str(Path(file_path).parent)
        if folder not in sys.path:
            sys.path.append(folder)
        spec = importlib.util.spec_from_file_location(module_string, file_path)
        module = importlib.util.module_from_spec(spec)
        sys.modules[module_string] = module
        spec.loader.exec_module(module)

    dyn_modules.append(module)

    return module


with time_line(f"Loading {len(files_canny_impls)} canny implementations"):
    for f in files_canny_impls:
        full_path = Path(dir_canny_impls) / f
        module = load_module(full_path, canny_impls)


2024-10-22 02:43:22.345 INFO root Loading 2 canny implementations started (notebook_cell:34)
2024-10-22 02:43:22.365 INFO root Loading rd_vec_v4_dibit started (notebook_cell:20)
2024-10-22 02:43:22.995 INFO root Loading rd_vec_v4_dibit took: 629.0782 ms (notebook_cell:20)
2024-10-22 02:43:23.058 INFO root Loading rd_numba_cuda_fp32 started (notebook_cell:20)
2024-10-22 02:43:23.060 INFO numba.cuda.cudadrv.driver init
2024-10-22 02:43:23.061 DEBUG numba.cuda.cudadrv.driver call driver api: cuInit
2024-10-22 02:43:23.073 DEBUG numba.cuda.cudadrv.driver call driver api: cuCtxGetCurrent
2024-10-22 02:43:23.074 DEBUG numba.cuda.cudadrv.driver call driver api: cuDeviceGetCount
2024-10-22 02:43:23.075 DEBUG numba.cuda.cudadrv.driver call driver api: cuDeviceGet
2024-10-22 02:43:23.075 DEBUG numba.cuda.cudadrv.driver call driver api: cuDeviceGetAttribute
2024-10-22 02:43:23.076 DEBUG numba.cuda.cudadrv.driver call driver api: cuDeviceGetAttribute
2024-10-22 02:43:23.077 DEBUG numba.cuda.cudadr

In [3]:
image_input_dir = "./image_input"
input_files = [
    f for f in os.listdir(image_input_dir) if f.endswith(".jpg") or f.endswith(".png")
]

images_color = []
images_gray = []

with time_line(f"Loading {len(input_files)} images"):
    for f in input_files:
        with time_line(f"Loading {f}"):
            file_image = Path(image_input_dir) / f
            image_color = cv2.imread(str(file_image), cv2.IMREAD_COLOR)
            images_color.append(image_color)
            # image_gray = (
            #    cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0
            # )
            image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
            images_gray.append(image_gray)

2024-10-22 02:44:04.960 INFO root Loading 2 images started (notebook_cell:9)
2024-10-22 02:44:04.986 INFO root Loading quinn.png started (notebook_cell:11)
2024-10-22 02:44:05.070 INFO root Loading quinn.png took: 82.7354 ms (notebook_cell:11)
2024-10-22 02:44:05.090 INFO root Loading park.png started (notebook_cell:11)
2024-10-22 02:44:05.973 INFO root Loading park.png took: 882.1270 ms (notebook_cell:11)
2024-10-22 02:44:05.974 INFO root Loading 2 images took: 1.0133 s (notebook_cell:9)


In [4]:
# Show a dropdown to select the image

output = widgets.Output()


def to_nbimage(image_buf, fmt="png", longest_side=None, use_widget=True):
    if len(image_buf.shape) == 3:
        is_color = True
    elif len(image_buf.shape) == 2:
        is_color = False
    else:
        raise ValueError(
            "The image does not have a valid shape. Expected either (height, width) or (height, width, channels)"
        )

    if image_buf.dtype == np.uint8:
        image_buf = image_buf.astype(np.float32) / 255.0
    else:
        image_buf = image_buf.astype(np.float32)

    height, width = image_buf.shape[:2]
    if longest_side is not None and (height > longest_side or width > longest_side):
        scale = longest_side / max(height, width)
        image_buf = cv2.resize(
            image_buf,
            (int(width * scale), int(height * scale)),
            interpolation=cv2.INTER_AREA,
        )

    if is_color:
        image_buf = (cv2.cvtColor(image_buf, cv2.COLOR_BGR2RGBA) * 255.0).astype(
            np.uint8
        )
    else:
        image_buf = (cv2.cvtColor(image_buf, cv2.COLOR_GRAY2RGBA) * 255.0).astype(
            np.uint8
        )

    f = BytesIO()
    pil_image = PILImage.fromarray(image_buf)
    pil_image.save(f, fmt)

    if use_widget:
        return widgets.Image(value=f.getvalue(), format=fmt)
    else:
        Image(data=f.getvalue())


@output.capture(clear_output=True, wait=True)
def on_menu_change(change=None):
    # Show the selected image
    image_color = images_color[image_dropdown.index]
    image_gray = images_gray[image_dropdown.index]

    height, width = image_gray.shape

    logging.info(f"Selected image: {image_dropdown.value} ({width}x{height})")
    logging.info(f"Selected canny implementation: {canny_impl_dropdown.value}")
    with time_line("Displaying input images"):
        display(
            widgets.HBox(
                [
                    to_nbimage(image_color, longest_side=300),
                    to_nbimage(image_gray, longest_side=300),
                ]
            )
        )

    canny_impl = next(
        canny_impl
        for canny_impl in canny_impls
        if canny_impl.__name__ == canny_impl_dropdown.value
    )

    with time_line("Blurring image"):
        image_blurred = canny_impl.blur_gauss(image_gray, sigma_slider.value)
    with time_line("Displaying blurred image"):
        display(to_nbimage(image_blurred, longest_side=300))


image_dropdown = widgets.Dropdown(
    options=input_files,
    description="Image",
    disabled=False,
)
canny_impl_dropdown = widgets.Dropdown(
    options=[canny_impl.__name__ for canny_impl in canny_impls],
    description="Canny Implementation",
    disabled=False,
    value=canny_impls[1].__name__,
)
sigma_slider = widgets.FloatSlider(
    value=3.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description="Sigma",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".1f",
)

image_dropdown.observe(on_menu_change, names="value")
canny_impl_dropdown.observe(on_menu_change, names="value")
sigma_slider.observe(on_menu_change, names="value")

display(
    widgets.VBox(
        [widgets.HBox([image_dropdown, canny_impl_dropdown, sigma_slider]), output]
    )
)
on_menu_change()

In [5]:
if False:
    cuda.select_device(0)
    cuda.synchronize()
    device = cuda.get_current_device()
    device.reset()
    gc.collect()